# 第3章: 役割の割り当て (ロールプロンプティング)

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## セットアップ

次のセットアップセルを実行して、APIキーをロードし、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
from botocore.exceptions import ClientError
import json

# utilsパッケージからhintsモジュールをインポート
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None):
    # 推論設定
    inference_config = {
        "temperature": 0.0,
        "maxTokens": 500
    }
    # 会話APIパラメータ
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"クライアントエラーが発生しました: {message}")

---

## レッスン

Claudeがあなたの言うこと以外に文脈を持たないというテーマを続けると、時には**Claudeに特定の役割を持たせる（必要な文脈をすべて含む）ことが重要です**。これは役割プロンプティングとも呼ばれます。役割の文脈に詳細を加えるほど、より良い結果が得られます。

**役割でClaudeをプライミングすることで、Claudeのパフォーマンスを向上させることができます**。これは、執筆からコーディング、要約に至るまで、さまざまな分野で効果があります。人間が「______のように考える」と言われると助けられることがあるのと似ています。役割プロンプティングは、Claudeの応答のスタイル、トーン、方法を変えることもできます。

**注意:** 役割プロンプティングは、システムプロンプト内で行われるか、ユーザーメッセージのターンの一部として行われることがあります。

### 例

以下の例では、役割プロンプトなしで、Claudeがスケートボードについての一文の視点を求められたとき、**率直でスタイルのない回答**を提供することがわかります。

しかし、Claudeに猫の役割を持たせると、Claudeの視点が変わり、したがって**Claudeの応答のトーン、スタイル、内容が新しい役割に適応します**。

**注意:** 使用できるボーナステクニックは、**Claudeにその意図された聴衆についてのコンテキストを提供すること**です。以下のように、プロンプトを調整して、Claudeが誰に話すべきかを伝えることもできます。「あなたは猫です」は「あなたはスケートボーダーの群衆に話している猫です」とはかなり異なる応答を生み出します。

ここに、システムプロンプトでの役割プロンプトなしのプロンプトがあります:

In [ ]:
# プロンプト
PROMPT = "In one sentence, what do you think about skateboarding?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

# ステップ
1. 提供されたマークダウンテキストを読み、プロンプトエンジニアリングの側面について議論します。
2. ドメイン固有の英語の用語、技術用語、およびバッククォート（`）で囲まれた用語を特定し、それらは英語のままにします。
3. 残りのテキストを日本語に翻訳し、意味が保持されるようにします。

# 注意
忠実に翻訳することに集中し、余分な出力やコメントを追加しないでください。

# 出力形式
指定された英語の用語をそのまま維持しながら、日本語で翻訳されたテキストを提供します。

In [ ]:
# System prompt
SYSTEM_PROMPT = "あなたは猫です。"

# Prompt
PROMPT = "一文で、スケートボードについてどう思いますか？"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

ロールプロンプティングを使用することで、Claudeに特定のスタイルで書かせたり、特定の声で話させたり、回答の複雑さを調整させたりすることができます。**ロールプロンプティングは、Claudeが数学や論理タスクをより良く実行するのにも役立ちます。**

例えば、以下の例では、明確な正解があり、それは「はい」です。しかし、Claudeはそれを間違えて情報が不足していると思い込んでいますが、実際には不足していません。

In [ ]:
# プロンプト
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

今、もし私たちが**Claudeを論理ボットとしてプライミングしたら**、それはClaudeの回答にどのように影響するでしょうか？

この新しい役割の割り当てによって、Claudeは正しい答えを出すことがわかりました。（ただし、すべての正しい理由からではないことは注目に値します）

In [ ]:
# System prompt
SYSTEM_PROMPT = "あなたは複雑な論理問題に答えるために設計された論理ボットです。"

# Prompt
PROMPT = "ジャックはアンを見ています。アンはジョージを見ています。ジャックは結婚していて、ジョージは結婚していませんが、アンが結婚しているかどうかはわかりません。結婚している人が結婚していない人を見ていますか？"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**注意:** このコースを通じて学ぶことは、**似たような結果を導き出すために使用できる多くのプロンプトエンジニアリング技術がある**ということです。どの技術を使用するかはあなたとあなたの好みによります！自分自身のプロンプトエンジニアリングスタイルを見つけるために、**実験することをお勧めします**。

上記のコンテンツを変更せずにレッスンのプロンプトを実験したい場合は、レッスンノートブックの一番下までスクロールして、[**Example Playground**](#example-playground)を訪れてください。

---

## 演習
- [演習 3.1 - 数学の修正](#exercise-31---math-correction)

### 演習 3.1 - 数学の修正
場合によっては、**Claudeは数学に苦労することがあります**、たとえそれが簡単な数学であっても。以下では、Claudeが数学の問題を正しく解決したと誤って評価していますが、2番目のステップに明らかな算術の間違いがあります。Claudeは実際にはステップごとに進む際にその間違いを見つけますが、全体の解決策が間違っているという結論には至りません。

`PROMPT`および/または`SYSTEM_PROMPT`を修正して、Claudeが解決策を正しく解決したのではなく、`incorrectly`解決したと評価するようにしてください。

In [ ]:
# System prompt - if you don't want to use a system prompt, you can leave this variable set to an empty string
SYSTEM_PROMPT = ""

# Prompt
PROMPT = """この方程式は正しく解かれていますか？

2x - 3 = 9
2x = 6
x = 3"""

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    if "incorrect" in text or "not correct" in text.lower():
        return True
    else:
        return False

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("この演習は正しく解かれました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_3_1_hint)

### おめでとうございます！

ここまでのすべての演習を解決した場合、次の章に進む準備が整いました。楽しいプロンプト作成を！

---

## 例のプレイグラウンド

これは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのように影響するかを見るためのエリアです。

In [ ]:
# プロンプト
PROMPT = "In one sentence, what do you think about skateboarding?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "あなたは猫です。"

# Prompt
PROMPT = "一文で、スケートボードについてどう思いますか？"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# プロンプト
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "あなたは複雑な論理問題に答えるために設計された論理ボットです。"

# Prompt
PROMPT = "ジャックはアンを見ています。アンはジョージを見ています。ジャックは結婚していて、ジョージは結婚していませんが、アンが結婚しているかどうかはわかりません。結婚している人が結婚していない人を見ていますか？"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))